<a href="https://colab.research.google.com/github/whopriyam/Benchmarking-Transformer-Networks-for-Sexual-Harassment-Detection-on-Twitter/blob/main/Models/Electra_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpletransformers
# Optional: for Experiment management
!pip install wandb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Sarcasm_Final_Dataset_Benchmark - Sheet1.csv')

In [4]:
df = df.filter(['Text','Label'], axis=1)
df

,Text,Label
0,donald trump insists he has the 'complete powe...,0
1,woman who admits to having watched golden glob...,1
2,eva longoria tans self out of visible spectrum,1
3,horrified pope calls philadelphia humanity's g...,1
4,the enigmatic art of josef koudelka,0
...,...,...
3994,god humbled to be the answer to 'jeopardy!' clue,1
3995,area woman worried she's forgetting what heath...,1
3996,"cupid cop gave out roses, cards on valentine's...",0
3997,new movie taps into nation's love of rapping k...,1


In [25]:
df["Label"].value_counts()

1    2299
0    1700
Name: Label, dtype: int64

In [5]:
train_df = df.sample(frac=0.7)
test_df = df.loc[~df.index.isin(train_df.index)]


train_final = train_df.sample(frac = 0.8)
eval_df = train_df.loc[~train_df.index.isin(train_final.index)]
len(eval_df), len(train_df) , len(test_df)

(560, 2799, 1200)

In [6]:
import pandas as pd
import pytest

from simpletransformers.classification import (
    ClassificationModel,
    # MultiLabelClassificationModel,
)


model_type = "electra"
model_name = "google/electra-small-discriminator"

In [8]:
 model = ClassificationModel(
        model_type,
        model_name,
        use_cuda=False,
        args={
            "num_train_epochs": 5,
            "no_save": True,
            "reprocess_input_data": True,
            "overwrite_output_dir": True,
            "scheduler": "constant_schedule",
            "max_seq_length": 20,
        },
    )

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['pooler.den

In [9]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2799 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/350 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/350 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/350 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/350 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/350 [00:00<?, ?it/s]

(1750, 0.2700251452896877)

In [10]:
len(test_df)

1200

In [11]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1384: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1200 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/150 [00:00<?, ?it/s]

In [12]:
list_eval = eval_df['Text'].to_list()
type(list_eval)

list

In [13]:
predictions, raw_outputs = model.predict(list_eval)

  0%|          | 0/560 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

In [14]:
from sklearn import metrics


In [26]:
targets = eval_df['Label'].to_list()
print(predictions)
print(targets)


[0 0 0 0 0 1 1 0 1 1 1 0 0 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 0 1 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 0 1 1 1 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 1 1 0 1 1 1 0 1 1 0 0 0 0 1 0 0 1 1 1 1 0
 1 1 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1
 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 0 0 1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 1 0 1
 0 1 1 0 1 1 1 1 0 1 1 0 0 1 1 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0 0 1 1 1 0 1 1
 1 1 1 0 0 1 0 1 0 0 0 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1 0 0 1 1 1 1
 1 0 1 1 1 0 1 0 1 1 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 0 1
 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 1 1
 1 0 0 0 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 0 0 0 1 1
 0 1 1 1 1 0 1 0 1 1 0 1 1 1 0 0 1 0 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 0
 0 0 0 1 0 1 1 0 0 1 1 0 0 0 1 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1
 1 1 1 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 0 0 1 1 0 1 0 1 0 1 0 1 0 0
 1 0 0 0 1 0 0 1 0 1 1 1 

In [27]:
metrics.accuracy_score(targets, predictions)

0.9946428571428572

In [28]:
print(metrics.classification_report(targets, predictions))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       235
           1       1.00      0.99      1.00       325

    accuracy                           0.99       560
   macro avg       0.99      1.00      0.99       560
weighted avg       0.99      0.99      0.99       560

